In [ ]:
import pandas as pd
import bs4 as bs
import zipfile
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
import os
import re

import win32com.client
from pywintypes import com_error
import sys, os
from os.path import isfile, join

import xml.etree.ElementTree as ET

## Input folder - Find if there is a twbx or twb file in the folder
- if there is a twbx, unzip it to create a twb, then work with this
- if there's only a twb, work with this

In [ ]:
input_path = "inputs"
output_path = "outputs"

mypath = "./{}".format(input_path)

In [ ]:
#only gets files and not directories within the inputs folder -https://stackoverflow.com/questions/3207219/how-do-i-list-all-files-of-a-directory
f = [f for f in os.listdir(mypath) if isfile(join(mypath, f))] 
f

In [ ]:
for i in [f for f in os.listdir(mypath) if isfile(join(mypath, f))] :
   
    if i.split(".")[-1] == 'twbx':
        packagedWorkbook = i
            
        with zipfile.ZipFile(input_path+"/"+packagedWorkbook, 'r') as zip_ref:
            zip_ref.extractall(input_path+"/")
    else:
        packagedWorkbook = ""
        
for i in [f for f in os.listdir(mypath) if isfile(join(mypath, f))] :
    #print(i.split(".")[-1])
    
    if i.split(".")[-1] == 'twb':
        unpackedWorkbook = i 
        

#print('\n')
print('packaged workbook: ' + packagedWorkbook)
print('unpackaged workbook: ' + unpackedWorkbook)

In [ ]:
tableauFile = input_path+"/"+unpackedWorkbook
tableauFile

# Parse xml to get all calculations

In [ ]:
tree = ET.parse(tableauFile)
root = tree.getroot()

In [ ]:
collator = []
calcNames = []
calcCaptions = []

for elem in root.findall("./datasources/datasource/column"):
    print(elem)
    
    dict_temp = {}
    
    if (elem.findall('calculation')) != []:    #only get nodes where there is a calculation
        try:
            print("caption: " + elem.attrib['caption'])
            dict_temp['caption'] = elem.attrib['caption']
            calcCaptions.append(elem.attrib['caption'])

            dict_temp['datatype'] = elem.attrib['datatype']
            dict_temp['name'] = elem.attrib['name']

            f2 = (elem.attrib['name']).replace(']','')
            f2 = f2.replace('[', '')
            calcNames.append(f2)
        except:
            print('skipped calc')
            print("caption: " + str(elem.attrib['name']))
            print("...........skipped calc..........")
        
        try:
            paramExists = elem.attrib['param-domain-type']
            print(paramExists)
            dict_temp['isParameter'] = 'yes'
            dict_temp['formula'] = 'NA'
        except:
            dict_temp['isParameter'] = 'no'
            
            try:
                for calc in elem.findall('calculation'):
                    dict_temp['formula'] = calc.attrib['formula']
            except:
                print('skipped calc')
                print("caption: " + str(elem.attrib['name']))
                print("...........skipped calc..........")
                dict_temp['formula'] = 'NA'
                
        else:
            print('NOT PROCESSING!!!!')
            print(elem.attrib['name'])
                
        collator.append(dict_temp)

In [ ]:
calcDict = dict(zip(calcNames, calcCaptions))
calcDict

In [ ]:
def default_to_friendly_names(formulaList):

    for i in formulaList:
        for tableauName, friendlyName in calcDict.items():
            i['formula'] = (i['formula']).replace(tableauName, friendlyName)
       
    return formulaList

In [ ]:
collator = default_to_friendly_names(collator)
collator

In [ ]:
df = pd.DataFrame(collator)

df.columns = ['CalculationName', 'DataType', 'Formula', 'isParameter', 'RawName']
df=df.sort_values(by=['isParameter','CalculationName'])
df=df.reset_index(drop=True)
df

In [ ]:
#df.to_csv(output_path+'/myExportedFormulas.csv', index = False)

## ........................ End of processing of all formulas ......................

# Getting all filters for all worksheets

In [ ]:
filters_in_sheet = []
context = []
collatelist = []

for worskheet in root.findall("./worksheets/worksheet"):
    
    tempdict = {}
    c = 0
    
    for filt in worskheet.findall('table/view/filter'):

        calcfromfilter = filt.attrib['column']        
        pat = '(?<=\:)(.*?)(?=\:)' 
        string_cleaned = calcfromfilter.split('].[')[1].replace(']','')
        
        tempdict['field'] = calcfromfilter
        tempdict['formula'] = calcfromfilter
        tempdict['counter'] = c
        tempdict['sheetname'] = worskheet.attrib['name']
        
        try:
            st1 = re.findall(pat,string_cleaned)[0]
            tempdict['field'] = st1
            tempdict['formula'] = st1
            collatelist.append(tempdict)
            
        except:
            st2 = string_cleaned.replace(':','')
            tempdict['field'] = st2
            tempdict['formula'] = st2
            collatelist.append(tempdict)

        try:
            tempdict['context'] = filt.attrib['context']
        except:
            tempdict['context'] = 'False'
           
        c = c + 1
        tempdict = {}
    
collatelist[0:5]

In [ ]:
collatelist = default_to_friendly_names(collatelist)
collatelist

In [ ]:
df1 = pd.DataFrame(collatelist)

df1 = df1[['sheetname', 'field','formula', 'context']]
df1.columns = ['Sheet Name', 'Field_RawName', 'Field_FriendlyName', 'Context filter']

df1

# Extracting rows and cols for each sheet

In [ ]:
collecteddata = []

for worksheet in root.findall("./worksheets/worksheet"):

    argumentstopass = ['rows', 'cols']
    
    for i in argumentstopass:   
    
        internaldict = {}

        internaldict['sheetname'] = worksheet.attrib['name']
        internaldict['type'] = i
        
        formulahere = worksheet.findall('table/'+i)[0].text
        internaldict['formula'] = formulahere
        
        collecteddata.append(internaldict)
    
collecteddata[0:5]

In [ ]:
for i in collecteddata:

    try:
        pattern = '\:.*?\:'
        pat = '(?<=\:)(.*?)(?=\:)'

        calculationsWithColon = re.findall(pattern,i['formula']) 
        calcsWithoutColon = []

        for n in calculationsWithColon:
            oneCalcWithoutColon = re.findall(pat,n)[0]

            calcsWithoutColon.append(oneCalcWithoutColon)

        i['extracted formulas'] = calcsWithoutColon
        
    except:
        i['extracted formulas'] = []
             
    newcalcs = []
    formulas_to_process = i['extracted formulas']
    
    for n in formulas_to_process:
        print(n)
        
        for tableauName, friendlyName in calcDict.items():
            
            n = n.replace(tableauName, friendlyName)
        newcalcs.append(n)
        
    i['processed formulas'] = newcalcs

In [ ]:
df2 = pd.DataFrame(collecteddata)
df2 = df2.drop(columns=['formula', 'extracted formulas'])
df2 = df2.pivot(index='sheetname', columns='type', values='processed formulas')
df2=df2.reset_index()
df2.head()

## Generating an excel file from a df, so the excel can be formatted, then turning the excel into a pdf

In [ ]:
#load env variable with full path to folder, as otherwise the pdf creation fails

with open("archive/path_string.txt", "r") as f:
    os.environ["path_string"] = f.read()
    
path_string = os.environ.get('path_string', 'Not Set')
path_string = r"{}".format(path_string)

- Loading the file names and output locations for the excel and pdfs to be produced

In [ ]:
name_to_use = 'RowsAndCols'

newFileName = 'outputs\{}'.format(name_to_use)
excelName = newFileName + ".xlsx"
pdfName = newFileName + ".pdf"
print(pdfName)

excel_path = path_string.format(excelName)
path_to_pdf = path_string.format(pdfName)
excel_path

- Functions to format the excel files

In [ ]:
def mainCol(colNumber):
    format_mainCol = workbook.add_format({'text_wrap': True, 'bold': True})
    format_mainCol.set_align('vcenter')
    format_mainCol.set_bg_color('#f4dfa4')
    format_mainCol.set_border(1)
    worksheet.set_column(colNumber,colNumber,20,format_mainCol)
    return worksheet

In [ ]:
def normalCol(colNumber, colWidth):
    format2 = workbook.add_format({'text_wrap': True})
    format2.set_align('vcenter')
    format2.set_border(1)
    worksheet.set_column(colNumber,colNumber,colWidth,format2)
    return worksheet

- Creation of excel file, setting each column with the desired format

In [ ]:
writer = pd.ExcelWriter(excelName, engine = 'xlsxwriter')

df2.to_excel(writer, sheet_name = 'Sheet1', index=False)
workbook=writer.book
worksheet = writer.sheets['Sheet1']

worksheet = mainCol(0)
worksheet = normalCol(1,20)
worksheet = normalCol(2,20)

worksheet.set_paper(9) #a4
worksheet.fit_to_pages(1,0) #fit to 1 page wide, n long

writer.save()

- Creation of pdf from excel file

In [ ]:
excel = win32com.client.Dispatch("Excel.Application")
excel.Visible = False

wb = excel.Workbooks.Open(excel_path)

ws_index_list=[1]
wb.Worksheets(ws_index_list).Select()
wb.ActiveSheet.ExportAsFixedFormat(0, path_to_pdf)
wb.Close()
excel.Quit()

In [ ]:
name_to_use = 'Filters'

newFileName = 'outputs\{}'.format(name_to_use)
excelName = newFileName + ".xlsx"
pdfName = newFileName + ".pdf"
print(pdfName)

excel_path = path_string.format(excelName)
path_to_pdf = path_string.format(pdfName)
excel_path

writer = pd.ExcelWriter(excelName, engine = 'xlsxwriter')

df1.to_excel(writer, sheet_name = 'Sheet1', index=False)
workbook=writer.book
worksheet = writer.sheets['Sheet1']

worksheet = mainCol(0)
worksheet = normalCol(1,20)
worksheet = normalCol(2,20)

worksheet.set_paper(9) #a4
worksheet.fit_to_pages(1,0) #fit to 1 page wide, n long

writer.save()

excel = win32com.client.Dispatch("Excel.Application")
excel.Visible = False

wb = excel.Workbooks.Open(excel_path)

ws_index_list=[1]
wb.Worksheets(ws_index_list).Select()
wb.ActiveSheet.ExportAsFixedFormat(0, path_to_pdf)
wb.Close()
excel.Quit()

In [ ]:
name_to_use = 'AllCalculations'

newFileName = 'outputs\{}'.format(name_to_use)
excelName = newFileName + ".xlsx"
pdfName = newFileName + ".pdf"
print(pdfName)

excel_path = path_string.format(excelName)
path_to_pdf = path_string.format(pdfName)
excel_path

writer = pd.ExcelWriter(excelName, engine = 'xlsxwriter')

df.to_excel(writer, sheet_name = 'Sheet1', index=False)
workbook=writer.book
worksheet = writer.sheets['Sheet1']

worksheet = mainCol(0)
worksheet = normalCol(1,10)
worksheet = normalCol(2,40)
worksheet = normalCol(3,10)
worksheet = normalCol(4,20)

worksheet.set_paper(9) #a4
worksheet.fit_to_pages(1,0) #fit to 1 page wide, n long

writer.save()

excel = win32com.client.Dispatch("Excel.Application")
excel.Visible = False

wb = excel.Workbooks.Open(excel_path)

ws_index_list=[1]
wb.Worksheets(ws_index_list).Select()
wb.ActiveSheet.ExportAsFixedFormat(0, path_to_pdf)
wb.Close()
excel.Quit()